## setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

In [ ]:
path = '/content/drive/MyDrive/python/2024-1학기/공공데이터 공모전/data/'
data = pd.read_json(path + 'mapping30.json')
hs = pd.read_json(path+'HS10전처리.json').explode('HS부호').reset_index(drop=True)
ex_other = pd.read_json(path+'ex_other_mapping.json')
label = pd.read_json(path+'handlabel.json')
vae = pd.read_json(path+'VAE.json').drop_duplicates('DSC').reset_index(drop=True)
rerank = pd.read_json(path+'Reranker.json')
rerank['DSC'] = data['DSC']

In [ ]:
for i in label[label['handlabel']=='none'].index:
  label['handlabel'][i] = []

In [ ]:
ex_other = ex_other.drop(['except_embedding', 'other_embedding'], axis=1)
data = pd.merge(data, vae[['DSC', 'VAE_top_10']], on='DSC', how='left')
data = pd.merge(data, rerank[['DSC', 'cross_result']], on='DSC', how='left')
data = pd.merge(data, ex_other, on='DSC', how='left')
data = data.drop(['transformer_embedding', 'maxscore', 'ex_maxscore', 'ot_maxscore'], axis=1)
data = data.fillna('0')
data

,DSC,DSC(import),LLMimport,matched,VAE_top_10,cross_result,except,other,ex_matched,ot_matched
0,"automotive repair shops, nec specialized auto...","automotive repair shops, nec specialized auto...","Brake linings, carburetors, radiators, fuel fi...","{'0.6494507475436267': [7320201000], '0.583288...","{'0.31902293069681525': [8421231000, 842131100...","{'1.51171875': [8511109000, 8511209000, 851130...",0,0,0,0
1,"general farms, primarily animals, nsk derives...","general farms, primarily animals, nsk derives...","Feeding equipment, veterinary supplies, and fa...","{'0.6073210918483304': [407190000, 407290000, ...","{'0.3268519493764378': [101211000], '0.3211919...","{'1.3984375': [1106209000, 1212999000], '1.182...",0,0,0,0
2,fish and seafoods the wholesale distribution ...,fish and seafoods the wholesale distribution ...,"Cooler units, pallets, crates, boxes, gloves, ...","{'0.6609429034116933': [2104102000], '0.650530...","{'0.44551670796837406': [308192000, 308292000,...","{'1.8134765625': [1604141019, 1604141029, 1604...",canned or packaged frozen,0,"{'0.603313268518764': [207250000], '0.59752304...",0
3,"new and used car dealers, nsk manufactures a ...","new and used car dealers, nsk manufactures a ...","Steel alloys for car chassis and body panels, ...","{'0.6839825584977619': [7320201000], '0.597469...","{'0.335585858008721': [8703218000], '0.3210952...","{'-1.0244140625': [7320201000], '-1.1787109375...",0,0,0,0
4,"automotive stampings, nsk manufacturing autom...","automotive stampings, nsk manufacturing autom...","Steel sheets, Aluminum alloys, Copper alloys, ...","{'0.6344416678820545': [7320201000], '0.630219...","{'0.2970822529666834': [7314310000, 7314410000...","{'0.46923828125': [8512900000, 8709900000], '-...",metal structural and container products,0,"{'0.6925433440102216': [7310290000], '0.687401...",0
...,...,...,...,...,...,...,...,...,...,...
1713,"printing and writing paper, nsk the wholesale...","printing and writing paper, nsk the wholesale...","Ink, toners, and printing supplies.","{'0.6356510586659162': [4821100000], '0.609555...","{'0.3877715229903089': [4821100000], '0.357612...","{'3.875': [4810131000, 4810191000], '3.0917968...",0,0,0,0
1714,leather and sheep-lined clothing manufacturin...,leather and sheep-lined clothing manufacturin...,"Leather hides, Sheep pelts or wool, Thread (sp...","{'0.7126465277629943': [6209201000, 6209301000...","{'0.43636181466719254': [6101302000, 610230200...","{'3.16015625': [6209201000, 6209301000, 620990...",fur apparel manufacture of leather clothesnonc...,0,"{'0.6939206024055644': [4203101010, 4203103010...",0
1715,"electronic parts and equipment, nec, nsk the ...","electronic parts and equipment, nec, nsk the ...","Integrated circuits, Microprocessors, Printed ...","{'0.6810985356731969': [8517719000], '0.664486...","{'0.3348703777042155': [8536691000], '0.331991...","{'4.11328125': [8518909000, 8522909000, 852990...",0,0,0,0
1716,"bottled and canned soft drinks, nsk manufactu...","bottled and canned soft drinks, nsk manufactu...","Glass or plastic bottles, Aluminum cans, Label...","{'0.6079746328721736': [3824996300], '0.580094...","{'0.3450108068097574': [2106901010], '0.344170...","{'0.2449951171875': [2201100000], '-0.63037109...",0,0,0,0


In [ ]:
def change_key_to_float(di):
  if di != '0':
    new = {}
    for key in di.keys():
      new[float(key)] = di[key]
    return new
  else:
    return '0'

In [ ]:
data['matched'] = data['matched'].apply(lambda x : change_key_to_float(x))
data['VAE_top_10'] = data['VAE_top_10'].apply(lambda x : change_key_to_float(x))
data['cross_result'] = data['cross_result'].apply(lambda x : change_key_to_float(x))
data['ex_matched'] = data['ex_matched'].apply(lambda x : change_key_to_float(x))
data['ot_matched'] = data['ot_matched'].apply(lambda x : change_key_to_float(x))

In [ ]:
def if_in_list(x, hslist):
  if x in hslist:
    return 1
  else:
    return 0

## function

In [ ]:
# 각 모델의 결과를 voting
def ensemble_mapping_vote(data, row, cut1, cut2, cut3):
  matched = list(set(sum([data['matched'][row][x] for x in list(data['matched'][row].keys())[:cut1]], [])))
  vae_matched = list(set(sum([data['VAE_top_10'][row][x] for x in list(data['VAE_top_10'][row].keys())[:cut2]], [])))
  rerank_matched = list(set(sum([data['cross_result'][row][x] for x in list(data['cross_result'][row].keys())[:cut3]], [])))
  # 다수결 : 3개 중 2개 이상이 추천한 것, 추천 결과의 intersection에 있는 코드만 추출
  match1 = list(set(matched) & set(vae_matched))
  match2 = list(set(rerank_matched) & set(vae_matched))
  match3 = list(set(rerank_matched) & set(matched))
  hslist = list(set(match1 + match2 + match3))
  if len(hslist)==0:
    return '0'
  else:
    tmp = hs.copy()
    tmp['filter'] = tmp['HS부호'].apply(lambda x : if_in_list(x, hslist))
    tmp = tmp[tmp['filter']==1].reset_index(drop=True)
    total = list(tmp['HS부호'])
    return total

In [ ]:
# except, other than에 해당하는 코드는 제거
def filter_except(data, row, cut1, cut2):
  hslist = list(set(data['total'][row]))
  if len(hslist)==0:
    return '0'
  else:
    tmp = hs.copy()
    tmp['filter'] = tmp['HS부호'].apply(lambda x : if_in_list(x, hslist))
    tmp = tmp[tmp['filter']==1].reset_index(drop=True)
    removelist = []
    try:
      exlist = list(set(sum([data['ex_matched'][row][x] for x in list(data['ex_matched'][row].keys())[:cut1]], [])))
      removelist = removelist + exlist
    except:
      pass
    try:
      otlist = list(set(sum([data['ot_matched'][row][x] for x in list(data['ot_matched'][row].keys())[:cut2]], [])))
      removelist = removelist + otlist
    except:
      pass
    try:
      filterlist = list(set(tmp['HS부호']) - set(removelist))
      tmp['filter'] = tmp['HS부호'].apply(lambda x : if_in_list(x, filterlist))
      tmp = tmp[tmp['filter']==1].reset_index(drop=True)
    except:
      pass
    total = list(tmp['HS부호'])
    return total

In [ ]:
def acc_score(data, label):
  x = pd.merge(label, data[['DSC', 'DSC(import)', 'total']], on='DSC', how='left')
  x['inter'] = '0'
  x['len'] = 0
  x['score'] = 0
  for i in range(len(x)):
    x['inter'][i] = len(set(x['handlabel'][i])&set(x['total'][i]))
    x['len'][i] = len(x['total'][i])
    if x['len'][i] > 0:
      x['score'][i] = x['inter'][i]/x['len'][i]
    else:
      x['score'][i] = 'No match'
  return x

In [ ]:
def show_total(data, row, show=False):
  dsc = data['DSC(import)'][row]
  hslist = data['total'][row]
  tmp = hs.copy()
  tmp['filter'] = tmp['HS부호'].apply(lambda x : if_in_list(x, hslist))
  tmp = tmp[tmp['filter']==1].reset_index(drop=True)
  if show==True:
    print('DSC(import) :', dsc)
    for text in list(set(tmp['품목명'])):
      print(text)
  return dsc, tmp

In [ ]:
# 각 모델의 결과를 union
def ensemble_mapping_union(data, row, cut1=1, cut2=1, cut3=1):
  matched = list(set(sum([data['matched'][row][x] for x in list(data['matched'][row].keys())[:cut1]], [])))
  vae_matched = list(set(sum([data['VAE_top_10'][row][x] for x in list(data['VAE_top_10'][row].keys())[:cut2]], [])))
  rerank_matched = list(set(sum([data['cross_result'][row][x] for x in list(data['cross_result'][row].keys())[:cut3]], [])))
  # 각 모델이 추천한 top1을 union
  hslist = list(set(matched + vae_matched + rerank_matched))
  if len(hslist)==0:
    return '0'
  else:
    tmp = hs.copy()
    tmp['filter'] = tmp['HS부호'].apply(lambda x : if_in_list(x, hslist))
    tmp = tmp[tmp['filter']==1].reset_index(drop=True)
    total = list(tmp['HS부호'])
    return total

## ensemble voting

---

- 코사인 유사도 : top10
- Rerank : top10
- VAE : top5

선정기준 : handlabel score

In [ ]:
result = data.copy()
result['total'] = '0'

In [ ]:
# 앙상블 보팅
for i in range(len(result)):
  result['total'][i] = ensemble_mapping_vote(result, i, 10, 5, 10)

## filtering



---

- except : top10 제거
- other than : top10 제거

In [ ]:
# 필터링
for i in range(len(result)):
  result['total'][i] = filter_except(result, i, 10, 10)

## Union

---

- 추천이 0개인 경우에 한하여 각 모델의 top1 추천을 union

In [ ]:
result['len'] = result['total'].apply(lambda x : len(x))
result[result['len']==0]

,DSC,DSC(import),LLMimport,matched,VAE_top_10,cross_result,except,other,ex_matched,ot_matched,total,len
77,mens and boys work clothing manufacturing men...,mens and boys work clothing manufacturing men...,"Thread, Buttons, Zippers, Fabric (cotton, poly...","{0.5804848584120496: [8420102000], 0.578735188...","{0.3003038786052023: [6116921000], 0.285664210...","{2.390625: [5807101000, 5807901000], 1.6904296...",fur apparel manufacture of workwearclothing ma...,0,"{0.7157792528964895: [6209201000, 6209301000, ...",0,[],0
426,"mens footwear, except athletic the production...","mens footwear, except athletic the production...","Leather for uppers, Sole materials such as rub...","{0.6777924554104988: [6404110000], 0.603888709...","{0.4151326616710438: [6404200000], 0.392267812...","{2.146484375: [5601290000, 5601309000, 5602900...",athletic the production of men's footwear des...,0,"{0.6952673822619286: [6404110000], 0.654042837...",0,[],0
738,mens and boys work clothing manufacturing men...,mens and boys work clothing manufacturing men...,"Thread, Zippers, Buttons, Snap fasteners, Velc...","{0.5952210755626861: [8420102000], 0.593003990...","{0.2862466342029036: [6116921000], 0.271220119...","{2.646484375: [5908001000], 2.380859375: [6209...",fur apparel manufacture of workwearclothing ma...,0,"{0.7157792528964895: [6209201000, 6209301000, ...",0,[],0
793,"commercial printing, nec commercial or job pr...","commercial printing, nec commercial or job pr...","Printing ink, Paper rolls, Engraving tools, Te...","{0.5960107198684106: [4911100000], 0.580748831...","{0.3061427128705968: [4821100000], 0.292477680...","{0.13037109375: [4801000000], -0.1595458984375...",screen and books,printing of newspapers and printing on labels ...,"{0.5585280054441162: [4903000000], 0.488215334...","{0.7041340763259429: [4821100000], 0.636120812...",[],0
894,"mens and boys trousers and slacks, nsk manufa...","mens and boys trousers and slacks, nsk manufa...","Woven fabrics, Knit fabrics, Zippers, Buttons,...","{0.6643512639834847: [4203101060, 4203103060],...","{0.31838451347547686: [4203101060, 4203103060]...","{3.564453125: [6111201000, 6111301000, 6111901...",fur apparel manufacture of other outerwearclot...,leather clothes and workwear manufacture of we...,"{0.7239910163887413: [6209201000, 6209301000, ...","{0.772926058196245: [4205001100], 0.7712074912...",[],0


In [ ]:
for i in list(result[result['len']==0].index):
  result['total'][i] = ensemble_mapping_union(result, i, 1, 1, 1)

In [ ]:
result['len'] = result['total'].apply(lambda x : len(x))
print(len(result[result['len']==0]))

0


## submission file

---

최종 결과물 excel file 생성

In [ ]:
submission = pd.read_excel(path + '비식별된 해외기업별 영문 텍스트데이터.xlsx')
submission = pd.merge(submission, result[['DSC', 'total']], on='DSC', how='left')[['ID', 'total']]
submission = submission.explode('total').reset_index(drop=True)
print(sum(submission['total'].isnull()))
submission = submission.dropna().reset_index(drop=True)
submission

0


,ID,total
0,1,8511109000
1,1,8511209000
2,1,8511309000
3,1,8511409000
4,1,8511509000
...,...,...
227568,10645,8479502000
227569,10645,8479902000
227570,10645,8508702000
227571,10645,9010901010


In [ ]:
submission['total'] = submission['total'].astype(str)
def hscode(x):
  if len(x)==9:
    x = '0' + x
    return x
  else:
    return x
submission['total'] = submission['total'].apply(lambda x : hscode(x))

In [ ]:
submission.to_excel(path+'submission.xlsx', index=False)

In [ ]:
pd.read_excel(path+'submission.xlsx')

,ID,total
0,1,8511109000
1,1,8511209000
2,1,8511309000
3,1,8511409000
4,1,8511509000
...,...,...
227568,10645,8479502000
227569,10645,8479902000
227570,10645,8508702000
227571,10645,9010901010
